# **Trading Bot**

> Studienarbeit in Applied Big Data Analytics. Umsetzung eines Trading-Bots mit Hilfe von Strategien und Auswertung von Wertpapieren.

*   **Einführung**
*   **Klassen**
*   **Funktionalität**
*   **Tests**

## **Einführung**

> Der Trading-Bot ist das Bindeglied zwischen Logik und Aktion. Die API liefert Strategien, die Signale für das Kaufen oder Verkaufen von Wertpapieren beinhalten. Mit diesen Signalen kann automatisch gehandelt werden.

*   **Vorteile**

    1.   Minimieren von Emotionen
    > Keine Emotionen beim Handeln, kein voreiliges Verkaufen

    2.   Disziplin
    > Bot hält sich an Vorgaben, kein Erzwingen von Profit möglich, Fehlkäufe unterbinden

    3.   Backtesting
    > Test mit historischen Daten, Test kann Fehler dezimieren

    4.   Schnelles Agieren
    > Trades werden SOFORT durchgeführt, erreichen von Stop-Loss oder Profit-Targets führen Trade aus

*   **Nachteile**

    1.   Systemfehler
    > Internetverbindung unterbrochen

    2.   Überwachung
    > System muss stets überwacht werden

    3.   Überoptimierung
    > Auf Papier großartige Strategie, Backtests zeigen Schwachstellen auf

    4.   Betrug
    > Oft sind Angebote für Bots zu gut um wahr zu sein!



## **Klassen**

> Nachfolgend werden alle Klassen, die in diesem Programm verwendet und gebraucht werden beschrieben.

### **User**

> Nutzer, der sich mit Hilfe dieses Bots am Tradingmarkt beteiligen möchte. Aktuell wird mit historischen Daten gehandelt. Ziel ist es, über eine API von *Bittrex* oder ähnlichem am Live-Markt Trades durchzuführen.

* **User** - id, Name, Guthaben

In [0]:
class User:
  def __init__(self, id, name, budget):
    self.id = id
    self.name = name
    self.budget = budget
    self.trades = []

  def getTradeId(self):
    return len(self.trades)

### **Strategie**

> Die API liefert fertige Strategien, die ein Signal und ein Wertpapier beinhalten. Mittels dieses Signals werden Trades geöffnet oder geschlossen.

* **Strategie** - id, Name, Wertpapier, Signal

In [0]:
class Strategie:
  def __init__(self, id, name, date, wp, signal):
    self.id = id
    self.name = name
    self.date = date
    self.wp = wp
    self.signal = signal

### **Trade**

> Ein Trade ist eine Aktion die am Handelsmarkt durchgeführt wird. Hier werden unter anderem das Datum des Trades, das dabei verwendete Wertpapier und die Menge, die gehandelt wurde angegeben.

* **Trade** - id, Datum des Trades, Wertpapier, Menge

In [0]:
class Trade:
  def __init__(self, id, date, wp, amount, status):
    self.id = id
    self.date = date
    self.wp = wp
    self.amount = amount
    self.status = status

  def getAmountInBTC(self):
    if self.wp == "BTC":
      return self.amount
    else:
      return (self.amount * float(coins[0].course))

### **Wertpapier**

> Wertpapiere sind die verschiedenen zur Verfügung stehenden Anlangen, wie z.B. *BTC* oder ähnliches.

* **Wertpapier** - id, Name, Shortcut, Datum, aktueller Kurs

In [0]:
class Wertpapier:
  def __init__(self, id, name, short, date, course):
    self.id = id
    self.name = name
    self.short = short
    self.date = date
    self.course = course

## **Funktionalität**

> Nachfolgend werden alle Funktionen, die das Auswerten der Strategien von der API ermöglichen und wie welche Signale umgesetzt werden, beschrieben.

### **Trends**

> Außerordentliche Entwicklung der Bewertung eines Wertpapiers in eine bestimmte Richtung.

* **Abwärtstrend**

> Ein Abwärtstrend ist dadurch gekennzeichnet, dass die Tiefs immer tiefer als das Tief zuvor und die Hochs immer tiefer als die Hochs zuvor sind. Die Extrempunkte sind also immer tiefer.

* **Aufwärtstrend**

> Ein Aufwärtstrend ist dadurch gekennzeichnet, dass die folgenden Hochs immer höher als das Hoch zuvor und die Tiefs ebenfalls immer höher als die Tiefs zuvor sind. Die Extrempunkte sind also immer höher.


### **Aktionen**

> Entsprechend der Trends und Signale, die die API liefert, werden entsprechende Aktionen ausgeführt.

* **Verkaufen**

> Beschreibung Verkaufen

Signal | Trend
--- | ---
**VO** | erstes Verkaufsignal
**V1** | zweites Verkaufsignal
**V2** | drittes Verkaufsignal
**V3** | viertes Verkaufsignal

* **Kaufen**

> Beschreibung Kaufen

Signal | Definition
--- | ---
**KO** | erstes Kaufsignal
**K1** | zweites Kaufsignal
**K2** | drittes Kaufsignal
**K3** | viertes Kaufsignal



* **Halten**

> Beschreibung Halten

Signal | Definition
--- | ---
**HO** | erstes Haltesignal
**H1** | zweites Haltesignal
**H2** | drittes Haltesignal
**H3** | viertes Haltesignal

---

In [0]:
### Kaufen von Menge x :: (Wertpapier) 

def buy(fak, wp):
  value = testUser.budget * fak
  testUser.budget = testUser.budget - value 
  today = date.today()
  tradeId = testUser.getTradeId()
  
  print(":: TRADE(" + str(tradeId) + ")")
  print(" :: BUY " + str(value) + " BTC of " + wp)
  newTrade = Trade(tradeId, today, wp, value, "run")
  testUser.trades.append(newTrade)

### Verkaufen von Menge x :: (Wertpapier) 

def sell(fak, wp):

  for trade in testUser.trades:
    atMarket = 0

    if (trade.status == "run"):
      print(":: TRADE(" + str(trade.id) + ") - " + trade.status)
      print(":: TRADE AMOUNT " + str(trade.getAmountInBTC()))
      if trade.wp == wp:
        atMarket = trade.amount
        testUser.budget = testUser.budget + atMarket
        print(" :: SELL(" + str(trade.id) + ") - in BTC: " + str(atMarket))
        testUser.trades[trade.id].status = "stop"

  print(":::: NEW BALANCE " + str(testUser.budget) + " BTC ::::")


def act(strategies):
  for strat in strategies:
    actWithStrat(strat)

def actWithStrat(strategie):

      ## VERKAUFEN

        #   - Anfordern des Trades um Höhe der auf dem Markt platzierten Tradeoffer zu erhalten
        #   - Einstellen der Verkaufsmenge
        #   - Verkaufen

      ## Verkaufen - 1. Signal
  if strategie.signal == "V0":
    sell(0.25, strategie.wp)

      ## Verkaufen - 2. Signal
  elif strategie.signal == "V1":
    sell(0.33, strategie.wp)

      ## Verkaufen - 3. Signal
  elif strategie.signal == "V2":
    sell(0.5, strategie.wp)

      ## Verkaufen - 4. Signal
  elif strategie.signal == "V3":
    sell(1, strategie.wp)

      ### KAUFEN

        #   - Anfordern des Budgets, welches dem User zur Verfügung steht
        #   - Einstellen der Kaufmenge
        #   - Erstellen der Tradeoffer
        #   - Eintragung in Trades des Users

      ## Kaufen - 1. Signal
  elif strategie.signal == "K0":
    buy(0.25, strategie.wp)

      ## Kaufen - 2. Signal
  elif strategie.signal == "K1":
    buy(0.33, strategie.wp)

      ## Kaufen - 3. Signal
  elif strategie.signal == "K2":
    buy(0.5, strategie.wp)

      ## Kaufen - 4. Signal
  elif strategie.signal == "K3":
    buy(1, strategie.wp)

  else:
    print(":::HALTEN")
    

## **Bot-Menu**

> Um Visualisierung zu ermöglichen wird ein kleines Menü erstellt, um Einstellungen und Strategien verwalten zu können.

In [0]:
einstellung1 = "1: Placeholder"
einstellung2 = "2: Placeholder"
einstellung3 = "3: Placeholder"
einstellungen = [einstellung1, einstellung2, einstellung3]

def printEinstellungen():
  print(":: EINSTELLUNGEN")
  for e in einstellungen:
    print(e)
  print()
  print("0: Zurück")
  print()

In [0]:
strategie1 = ["1: Strategie #1", "123"]
strategie2 = ["2: Strategie #2", "234"]
strategie3 = ["3: Strategie #3", "345"]
strategien = [strategie1, strategie2, strategie3]

def printStrategien():
  print(":: STRATEGIEN")
  for s in strategien:
    print(s[0])
  print()
  print("0: Zurück")
  print()

def runStrategie(id):
  strat = getStrategie(strategien[id-1][1])

  actWithStrat(strat)

In [0]:
budget1 = "1: Guthaben"
budget2 = "2: Einzahlung"
budget3 = "3: Zahlungsverlauf"
budgets = [budget1, budget2, budget3]

def printBudgets():
  print(":: BUDGET")
  for b in budgets:
    print(b)
  print()
  print("0: Zurück")
  print()


In [0]:
menu1 = "1: Einstellungen"
menu2 = "2: Strategien"
menu3 = "3: Budget"

menu = [menu1, menu2, menu3]

def printMenu():
  print(":: TRADING BOT")
  for m in menu:
    print(m)
  print()

## **Tests**

> Beschreibung Tests

In [1020]:
!git clone https://github.com/sleuoth-hof/trading_2019.git

fatal: destination path 'trading_2019' already exists and is not an empty directory.


In [0]:
from datetime import date
import csv

coins = []

with open('trading_2019/TradingBot/Code/v1/coins_1.csv', 'r') as _filehandler:
  csv_file_reader = csv.DictReader(_filehandler)

  for row in csv_file_reader:
    coin = Wertpapier(row['id'], row['name'], row['short'], row['date'], row['course'])
    coins.append(coin)
  

In [0]:
def getStratCSV():
  with open('trading_2019/TradingBot/Code/v1/strats.csv', 'r') as _filehandler:
    csv_file_reader = csv.DictReader(_filehandler)

    strategies = []

    for row in csv_file_reader:
      strat = Strategie(row['Id'], row[' name'], row[' date'], row[' wp'], row[' signal'])
      strategies.append(strat)
    
    return strategies

In [1023]:
for coin in coins:
  print(":: " + coin.name + " (" + coin.short +")" + " am: " + coin.date + " - " + coin.course)

:: Bitcoin (BTC) am: 20191214 - 7065.3398
:: Ethereum (ETH) am: 20191214 - 141.4040
:: Ripple (XRP) am: 20191214 - 0.2153
:: Tether (USDT) am: 20191214 - 1.0066
:: Litecoin (LTC) am: 20191214 - 43.1805


In [0]:
def fetchStrategies():
  print(":: FETCHING - get pending strategies")
  return getStratCSV()

## TEST 1

* *Max Mustermann, Startbudget: 1 BTC*
  * Strategien
    * Strategie 1 : K1
    * Strategie 2 : V1
  * Funktionen
    * Handel von Bitcoins mit Bitcoins
    * Kaufen nach Strategie K1
    * Verkaufen nach Strategie V1

In [1025]:
testUser = User(0, "Max Mustermann", 1)

allStrategies = fetchStrategies()

act(allStrategies)

:: FETCHING - get pending strategies
:: TRADE(0)
 :: BUY 0.33 BTC of BTC
:: TRADE(1)
 :: BUY 0.6699999999999999 BTC of ETH
:: TRADE(2)
 :: BUY 0.0 BTC of USDT
:: TRADE(0) - run
:: TRADE AMOUNT 0.33
 :: SELL(0) - in BTC: 0.33
:: TRADE(1) - run
:: TRADE AMOUNT 4733.777665999999
:: TRADE(2) - run
:: TRADE AMOUNT 0.0
:::: NEW BALANCE 0.33 BTC ::::
:: TRADE(1) - run
:: TRADE AMOUNT 4733.777665999999
 :: SELL(1) - in BTC: 0.6699999999999999
:: TRADE(2) - run
:: TRADE AMOUNT 0.0
:::: NEW BALANCE 1.0 BTC ::::


In [1026]:
printMenu()

printEinstellungen()

printStrategien()

printBudgets()

:: TRADING BOT
1: Einstellungen
2: Strategien
3: Budget

:: EINSTELLUNGEN
1: Placeholder
2: Placeholder
3: Placeholder

0: Zurück

:: STRATEGIEN
1: Strategie #1
2: Strategie #2
3: Strategie #3

0: Zurück

:: BUDGET
1: Guthaben
2: Einzahlung
3: Zahlungsverlauf

0: Zurück

